# Single arm motion planning

This notebook shows how to use the SingleArmOmplPlanner with Drake to plan a collision-free path between two joint configurations.

In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from cloth_tools.drake.building import add_meshcat_to_builder, finish_build
from cloth_tools.drake.scenes import add_ur5e_and_table_to_builder
from cloth_tools.drake.visualization import publish_joint_path
from cloth_tools.ompl.single_arm_planner import SingleArmOmplPlanner

## 1. Setting up Drake for collision checking 

### 1.1 Start building the Drake diagram

Building Drake diagram can be quite complex. 
To simplify this Drake has a helper class called `RobotDiagramBuilder` that helps linking all the Drake systems.

Using this class and its attirbutes we can easily add URDFs and weld them together.
As far as I know, a weld is exactly the same as a fixed joint in URDF.

In [ ]:
robot_diagram_builder = RobotDiagramBuilder()  # time_step=0.001 even when I set timestep I get the mimic joint warning
meshcat = add_meshcat_to_builder(robot_diagram_builder)
arm_index, gripper_index = add_ur5e_and_table_to_builder(robot_diagram_builder)

### 1.2 Finalize the diagram

In [ ]:
diagram, context = finish_build(robot_diagram_builder, meshcat)
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

### 1.3 Creating the collision checker

In [ ]:
collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[arm_index, gripper_index],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

In [ ]:
start_joints = np.deg2rad([0, -90, -90, -90, 90, 0])
collision_checker.CheckConfigCollisionFree(start_joints)

In [ ]:
# To visualize the start pose in meshcat
plant.SetPositions(plant_context, arm_index, start_joints)
diagram.ForcedPublish(context)

In [ ]:
goal_joints = np.deg2rad([180, -135, 95, -50, -90, -90])
collision_checker.CheckConfigCollisionFree(goal_joints)

## 2. Integration with OMPL

For integration with OMPL, we provide a `SingleArmOmplPlanner`, which handles conversion to OMPL data types and creates OMPL objects. 

The only "external" thing this class requires, a function tells it whether a given joint configuration is valid or not.

In this case, the `CheckConfigCollisionFree` method of the `SceneGraphCollisionChecker` we made can be used for this purpose.

In [ ]:
planner = SingleArmOmplPlanner(is_state_valid_fn=collision_checker.CheckConfigCollisionFree)

# You can choose the planner OMPL uses like so:
# import ompl.geometric as og
# simple_setup = planner._simple_setup
# simple_setup.setPlanner(og.LBKPIECE1(simple_setup.getSpaceInformation()))

In [ ]:
path = planner.plan_to_joint_configuration(start_joints, goal_joints)

## 3. Visualizing the path in Drake

Run the cell below and look at Meshcat in your browser to see your robot moving.

In [ ]:
from cloth_tools.path.execution import calculate_path_array_duration


duration = calculate_path_array_duration(np.array(path))
print(duration)

publish_joint_path(path, duration, meshcat, diagram, context, arm_index)

In [ ]:
path[0]

In [ ]:
from typing import List, Tuple

from airo_typing import JointConfigurationType
from pydrake.trajectories import PiecewisePolynomial, Trajectory
from pydrake.multibody.optimization import CalcGridPointsOptions, Toppra


def time_parametrize(path: List[JointConfigurationType], duration) -> Tuple[Trajectory, Trajectory]:
    # original path q(s) with s = s(t) hence q(s(t))
    plant = diagram.plant()
    # start_time = 0
    # end_time = duration

    # q_traj = PiecewisePolynomial.CubicWithContinuousSecondDerivatives(
    #     np.linspace(start_time, end_time, len(path)), path_np.T
    # )

    path_array = np.array(path)

    if len(path_array) >= 3:
        q_traj = PiecewisePolynomial.CubicWithContinuousSecondDerivatives(
            np.linspace(0.0, 1.0, len(path_array)), path_array.T
        )
    else:
        q_traj = PiecewisePolynomial.FirstOrderHold([0.0, 1.0], path_array.T)

    gridpoints = Toppra.CalcGridPoints(q_traj, CalcGridPointsOptions())
    toppra = Toppra(q_traj, plant, gridpoints)
    toppra.AddJointAccelerationLimit([-1.2] * 6, [1.2] * 6)
    toppra.AddJointVelocityLimit([-1] * 6, [1] * 6)
    t_traj = toppra.SolvePathParameterization()

    return q_traj, t_traj


q_traj, t_traj = time_parametrize(path, duration)


from typing import List, Tuple

import numpy as np
from airo_typing import JointConfigurationType
from loguru import logger
from pydrake.geometry import Cylinder, Meshcat, Rgba
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.multibody.tree import ModelInstanceIndex
from pydrake.planning import RobotDiagram
from pydrake.systems.framework import Context


def publish_q_traj_t_traj(
    q_traj: Trajectory,
    t_traj: Trajectory,
    meshcat: Meshcat,
    diagram: RobotDiagram,
    context: Context,
    arm_index: ModelInstanceIndex,
) -> None:
    plant = diagram.plant()
    plant_context = plant.GetMyContextFromRoot(context)

    meshcat.StartRecording(set_visualizations_while_recording=False)

    duration = t_traj.end_time()
    fps = 60.0
    frames = duration * fps

    for t in np.linspace(0, duration, int(np.ceil(frames))):
        context.SetTime(t)
        q = q_traj.value(t_traj.value(t).item())
        plant.SetPositions(plant_context, arm_index, q)
        diagram.ForcedPublish(context)

    meshcat.StopRecording()
    meshcat.PublishRecording()


publish_q_traj_t_traj(q_traj, t_traj, meshcat, diagram, context, arm_index)

In [ ]:
t_traj.value(0)

In [ ]:
t_traj.value(0).item(), t_traj.end_time()

In [ ]:
%%timeit
time_parametrize(path, duration)

In [ ]:
# def execute_traj(robot, q_traj: Trajectory, t_traj: Trajectory):
#     # TODO: write and put in seperate reusable module
#     # sample trajectory every 0.005 seconds until end_time()
#     control_freq = 0.005
#     t = 0.0
#     while t < t_traj.end_time():
#         joint_config_at_t = q_traj.value(t_traj.value(t))[0:6]
#         robot.servo_to_joint_configuration(joint_config_at_t, control_freq).wait()
#         t += control_freq

#     robot.rtde_control.servoStop(1.0)

In [ ]:
path_array = np.array(path)
# PiecewisePolynomial.CubicWithContinuousSecondDerivatives(np.linspace(0.0, 1.0, len(path_array)), path_array.T)